# Systems genetics 2020 - Final Project  

#### Note!
Our project is devided into several python modules which are loaded and excecuted from this notebook. <br>
Most of our code's project can be found under the /modules directory. 

Import public packges

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f
import matplotlib.pyplot as plt
import os.path
import sys
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Import custom packges (our python modules)

In [2]:
sys.path.append('modules')

from utils import print_dim, print_stats
from preprocessing import data_annotations_merge
from regression import eqtl_analysis, qtl_analysis
from results_analysis import get_associations, cis_trans_annotation

In [3]:
bold_s = '\033[1m'
bold_e = '\033[0m'

Load data files

In [4]:
liver_exp = pd.read_csv('data/GSE17522_series_matrix_liver.txt', sep = "\t")
brain_exp = pd.read_csv('data/GSE36674_series_matrix_hypothalamus.txt', sep = "\t")

liver_annotations = pd.read_csv('data/annotations_liver_GPL6466-9752.txt', sep = "\t")
brain_annotations = pd.read_csv('data/annotation_brain.annot', sep = "\t")

genotypes = pd.read_excel('data/genotypes.xls', headers=None)
phenotypes = pd.read_excel('data/phenotypes.xls')

mgi = pd.read_csv('data/MGI_Coordinates.Build37.rpt.txt', sep = "\t", error_bad_lines=False, warn_bad_lines=False)

### 2. Gene expression data preprocessing

#### A. Merge data file with annotation file to get your input matrix

Brain

In [5]:
brain_annotations = brain_annotations.rename(columns={'Gene symbol' : 'GENE_SYMBOL'})
brain_matrix = data_annotations_merge(brain_exp, brain_annotations)

Liver

In [6]:
liver_matrix = data_annotations_merge(liver_exp, liver_annotations)

# Keep BXD columns and identifier columns only
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)
id_cols = ['ID', 'GENE_SYMBOL']
liver_matrix = liver_matrix[id_cols + liver_bxd_cols]

# Rename BXD columns
liver_matrix = liver_matrix.rename(columns={col : col.split('_')[1] + '_' + col.split('_')[2] \
                                            for col in liver_matrix.columns.drop(id_cols)})

#### B. Keep only BXD columns that exists in the four files: genotypes, phenotypes and each tissue seperately

In [7]:
genotypes_bxd_cols = list(genotypes.filter(regex=("BXD*")).columns)
phenotypes_bxd_cols = list(phenotypes.filter(regex=("BXD*")).columns)

genotypes_baseline_cols = ["Locus", "Chr_Build37", "Build37_position"]
phenotypes_baseline_cols = ["Phenotype", "Authors", "Year", "Pubmed Id"]

# liver common columns
liver_matrix = liver_matrix.rename(columns = {col : col.split('_')[0] for col in liver_matrix.drop(columns = id_cols)}) 
liver_bxd_cols = list(liver_matrix.filter(regex='BXD').columns)
liver_common_bxd = list(set(genotypes_bxd_cols) & set(phenotypes_bxd_cols) &  set(liver_bxd_cols))
liver_common_bxd.sort()
liver_matrix = liver_matrix[['GENE_SYMBOL'] + liver_common_bxd]
liver_genotypes = genotypes[genotypes_baseline_cols + liver_common_bxd]
liver_phenotypes = phenotypes[phenotypes_baseline_cols + liver_common_bxd]

# brain common columns
brain_matrix = brain_matrix.rename(columns = {col : col.split('_')[0] for col in brain_matrix.drop(columns = id_cols)}) 
brain_bxd_cols = list(brain_matrix.filter(regex='BXD').columns)
brain_common_bxd = list(set(genotypes_bxd_cols) & set(phenotypes_bxd_cols) &  set(brain_bxd_cols))
brain_common_bxd.sort()
brain_matrix = brain_matrix[['GENE_SYMBOL'] + brain_common_bxd]
brain_genotypes = genotypes[genotypes_baseline_cols + brain_common_bxd]
brain_phenotypes = phenotypes[phenotypes_baseline_cols + brain_common_bxd]

# drop empty lines from phenotypes file
brain_phenotypes = brain_phenotypes.iloc[brain_phenotypes[brain_common_bxd].dropna(how='all').index]
liver_phenotypes = liver_phenotypes.iloc[liver_phenotypes[liver_common_bxd].dropna(how='all').index]

#### C. Use only representative genomic loci - Drop duplicated rows (of neighboring loci)

In [8]:
# liver genotypes filtering
snps_org = len(liver_genotypes)

bxd_data = liver_genotypes[liver_common_bxd]
duplicates_indx = bxd_data[bxd_data.shift() != bxd_data].dropna(how='all').index
liver_genotypes = liver_genotypes.loc[duplicates_indx]

print("LIVER Genotypes: Drop duplications:\n#SNPs before: %d   -->   #SNPs after: %d" % (snps_org, len(liver_genotypes)))
liver_genotypes.head(5)

# Brain genotypes filtering
snps_org = len(brain_genotypes)

bxd_data = brain_genotypes[brain_common_bxd]
duplicates_indx = bxd_data[bxd_data.shift() != bxd_data].dropna(how='all').index
brain_genotypes = brain_genotypes.loc[duplicates_indx]

print("LIVER Genotypes: Drop duplications:\n#SNPs before: %d   -->   #SNPs after: %d" % (snps_org, len(brain_genotypes)))
brain_genotypes.head(5)

LIVER Genotypes: Drop duplications:
#SNPs before: 3796   -->   #SNPs after: 1403
LIVER Genotypes: Drop duplications:
#SNPs before: 3796   -->   #SNPs after: 1598


,Locus,Chr_Build37,Build37_position,BXD1,BXD100,BXD101,BXD102,BXD103,BXD11,BXD12,...,BXD80,BXD83,BXD84,BXD85,BXD87,BXD89,BXD90,BXD95,BXD97,BXD99
0,rs6269442,1,3482276,B,B,U,U,U,B,D,...,D,H,B,D,B,B,B,D,B,B
2,rs6376963,1,5008090,B,B,U,U,U,B,D,...,D,H,B,D,B,B,B,D,B,B
3,rs3677817,1,5176059,B,B,U,U,U,B,D,...,D,H,B,D,B,B,B,D,B,B
4,rs8236463,1,5579194,B,B,U,U,U,D,D,...,D,H,B,D,B,B,B,D,B,B
6,rs6298633,1,6820242,B,B,U,U,U,D,D,...,D,H,B,D,B,B,B,D,B,B


#### • Remove rows with no gene identifier, <br> • Remove rows with low maximal value.  <br> • Remove rows with low variance.  <br> • Average multiple rows


Liver

In [9]:
max_treshold = 0.7
var_treshold = 0.2

# Remove rows with no gene identifier
liver_matrix = liver_matrix[~liver_matrix['GENE_SYMBOL'].isna()]
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)

for col in liver_bxd_cols:
    liver_matrix[col] = liver_matrix[col].astype('float64')

# Filter by maximal value
liver_matrix['max'] = liver_matrix.drop(columns=['GENE_SYMBOL']).max(axis=1)
liver_matrix = liver_matrix[liver_matrix['max'] >= max_treshold]
liver_matrix = liver_matrix.drop(columns = 'max')
print("Num of rows after removing all rows with maximal values less than %.2f: %d " % (max_treshold, len(liver_matrix)))

# Filter by variance
liver_matrix['var'] = liver_matrix.drop(columns=['GENE_SYMBOL']).var(axis=1)
liver_matrix = liver_matrix[liver_matrix['var'] >= var_treshold]
liver_matrix = liver_matrix.drop(columns = 'var')
print("Num of rows after removing all rows with variance less than %.2f: %d " % (var_treshold, len(liver_matrix)))

# Group multiple rows by mean
liver_matrix = liver_matrix.groupby('GENE_SYMBOL').agg('mean').reset_index()
print("Num of rows after removing duplicated rows: %d ", len(liver_matrix))

Num of rows after removing all rows with maximal values less than 0.70: 7772 
Num of rows after removing all rows with variance less than 0.20: 1308 
Num of rows after removing duplicated rows: %d  1285


Brain

In [10]:
max_treshold = 9
var_treshold = 0.04

# Remove rows with no gene identifier
brain_matrix = brain_matrix[~brain_matrix['GENE_SYMBOL'].isna()]
brain_bxd_cols = list(brain_matrix.filter(regex=("BXD*")).columns)

for col in brain_bxd_cols:
    brain_matrix[col] = brain_matrix[col].astype('float64')

# Filter by maximal value
brain_matrix['max'] = brain_matrix.drop(columns=['GENE_SYMBOL']).max(axis=1)
brain_matrix = brain_matrix[brain_matrix['max'] >= max_treshold]
brain_matrix = brain_matrix.drop(columns = 'max')
print("Num of rows after removing all rows with maximal values less than %.2f: %d " % (max_treshold, len(brain_matrix)))

# Filter by variance
brain_matrix['var'] = brain_matrix.drop(columns=['GENE_SYMBOL']).var(axis=1)
brain_matrix = brain_matrix[brain_matrix['var'] >= var_treshold]
brain_matrix = brain_matrix.drop(columns = 'var')
print("Num of rows after removing all rows with variance less than %.2f: %d " % (var_treshold, len(brain_matrix)))

# Group multiple rows by mean
brain_matrix = brain_matrix.groupby('GENE_SYMBOL').agg('mean').reset_index()
print("Num of rows after removing duplicated rows: %d ", len(brain_matrix))

Num of rows after removing all rows with maximal values less than 9.00: 9893 
Num of rows after removing all rows with variance less than 0.04: 1444 
Num of rows after removing duplicated rows: %d  1247


#### • Average across different individuals of the same strain (Females and males)

In [11]:
liver_matrix = liver_matrix.rename(columns = {col : col.split('_')[0] for col in liver_matrix.drop(columns = ['GENE_SYMBOL'])}) 
liver_matrix = liver_matrix.set_index(['GENE_SYMBOL']) 
liver_matrix = liver_matrix.groupby(by=liver_matrix.columns, axis=1).mean()
liver_matrix = liver_matrix.reset_index()

brain_matrix = brain_matrix.rename(columns = {col : col.split('_')[0] for col in brain_matrix.drop(columns = ['GENE_SYMBOL'])}) 
brain_matrix = brain_matrix.set_index(['GENE_SYMBOL']) 
brain_matrix = brain_matrix.groupby(by=brain_matrix.columns, axis=1).mean()
brain_matrix = brain_matrix.reset_index()

### 3. eQTL analysis

#### Run  Regression

In [12]:
print_dim(len(liver_matrix), len(liver_genotypes), title="Liver: ")
print_dim(len(brain_matrix), len(brain_genotypes), title="Brain: ")

Liver: 
* Expression matrix size: 1285
* Genotype matrix size: 1403
* Expected num of tests: 1,802,855

Brain: 
* Expression matrix size: 1247
* Genotype matrix size: 1598
* Expected num of tests: 1,992,706



In [13]:
# Brain eqtl
if not os.path.exists("brain_reg_results.csv"):
    brain_eqtls = eqtl_analysis(brain_matrix, brain_genotypes, file_prefix="brain_")
else:
    brain_eqtls = pd.read_csv("brain_reg_results.csv")
    brain_eqtls = brain_eqtls.drop(columns='Unnamed: 0')
    
# Liver eqtl
if not os.path.exists("liver_reg_results.csv"):
    liver_eqtls = eqtl_analysis(liver_matrix, liver_genotypes, file_prefix="liver_")
else:
    liver_eqtls = pd.read_csv("liver_reg_results.csv")
    liver_eqtls = liver_eqtls.drop(columns='Unnamed: 0')

#### Cis/Trans annotation

In [14]:
brain_eqtls = cis_trans_annotation(brain_eqtls.copy(), mgi)
liver_eqtls = cis_trans_annotation(liver_eqtls.copy(), mgi)

100%|██████████████████████████████████████████████████████████████████████████████| 1285/1285 [05:09<00:00,  4.15it/s]


#### Multiple test correction and associations filtering

In [15]:
brain_associations, brain_num_tests = get_associations(brain_eqtls) # by bonfferoni corrections
brain_associations.to_csv("brain_assoc_eqtl.csv")

liver_associations, liver_num_tests = get_associations(liver_eqtls) # by bonfferoni corrections
liver_associations.to_csv("brain_assoc_eqtl.csv")

print_stats(brain_associations, brain_num_tests, "1. Brain")
print_stats(liver_associations, liver_num_tests, "\n\n2. Liver")

1. Brain
Number of tests:  1992706

Number of different significant eQTLs:  472
From which: 
 260 - cis-acting 
 231 - trans-acting 

Number of total significant eQTLs:  868
From which: 
 349 - cis-acting 
 337 - trans-acting 


2. Liver
Number of tests:  1802855

Number of different significant eQTLs:  481
From which: 
 261 - cis-acting 
 278 - trans-acting 

Number of total significant eQTLs:  840
From which: 
 325 - cis-acting 
 468 - trans-acting 


### 4. QTL analysis

* Currently looks like no morphine phenotype is significant..

In [16]:
# Get all phenotypes related to Morphine by naive text search
morphine_phenotypes_b = brain_phenotypes[brain_phenotypes['Phenotype'].str.contains("Morphine")]
morphine_phenotypes_l = liver_phenotypes[liver_phenotypes['Phenotype'].str.contains("Morphine")]

In [17]:
logpval_brain = qtl_analysis(morphine_phenotypes_b, brain_genotypes, file_prefix="brain_morphine_")
logpval_liver = qtl_analysis(morphine_phenotypes_l, liver_genotypes, file_prefix="liver_morphine_")

100%|████████████████████████████████████████████████████████████████████████████████| 115/115 [10:43<00:00,  5.60s/it]


In [18]:
assoc_qtl_l, ntests_qtl_l = get_associations(logpval_brain) # by bonfferoni corrections
assoc_qtl_b, ntests_qtl_b = get_associations(logpval_liver) # by bonfferoni corrections

### 6. Causality Test

In [19]:
import math

def get_L_dependent_dist_params(df, A, l):
    """ Calculate the parameters of the distribution A|L=l.
        L can get 0 or 1
        The event 'A' can be a complex trait (C) or a gene expression (R)
    """
    obs = df[df.L == l][A]
    mu = obs.mean()
    var = obs.var(ddof=0)
    
    df[A + '|L_mean'][df.L == l] = mu
    df[A + '|L_var'][df.L == l] = var
    return df

def get_columns_params(col_a, col_b):
    # column A parameters
    mu_a = col_a.mean()
    var_a = col_a.var(ddof=0)
    # column B parameters
    mu_b = col_b.mean()
    var_b = col_b.var(ddof=0)
    # Correlation coef. between A and B
    corr = col_a.corr(col_b)
    return mu_a, var_a, mu_b, var_b, corr


def get_conditional_dist_params(mu_a, var_a, mu_b, var_b, corr, b_i):
    """ Calculate the parameters of the conditional distribution A|B (in our case it could be C|R or R|C)  
        The calculation is a bit different compared to the conditional probability given L
    """
    std_a = np.sqrt(var_a)
    std_b = np.sqrt(var_b)
    mu = mu_a + corr*std_a/std_b * (b_i - mu_b)
    var = var_a * (1-np.square(corr))
    return mu, var  

def norm_pdf(x, mean, var):
    denom = np.sqrt(2 * math.pi * var)
    num = math.exp(-np.square(x-mean)/(2*var))
    return (num/denom)

def init_models(input_table):
    m1_df = pd.DataFrame(columns=['Individual', 'L', 'R', 'C', 
                                   'R|L_mean', 'R|L_var', 
                                   'C|R_mean', 'C|R_var',
                                   'P(R|L)', 'P(C|R)',
                                   'P(L) * P(C|R) * P(R|L)'])
    m1_df[['Individual', 'L', 'R', 'C']] = input_table

    m2_df = pd.DataFrame(columns=['Individual', 'L', 'R', 'C', 
                                   'C|L_mean', 'C|L_var', 
                                   'R|C_mean', 'R|C_var',
                                   'P(C|L)', 'P(R|C)',
                                   'P(L) * P(R|C) * P(C|L)'])
    m2_df[['Individual', 'L', 'R', 'C']] = input_table

    m3_df = pd.DataFrame(columns=['Individual', 'L', 'R', 'C', 
                                   'R|L_mean', 'R|L_var', 
                                   'C|L_mean', 'C|L_var',
                                   'P(R|L)', 'P(C|L)',
                                   'P(L) * P(R|L) * P(C|L)'])
    m3_df[['Individual', 'L', 'R', 'C']] = input_table
    
    return m1_df, m2_df, m3_df

def generate_models_df(input_table):
    # Three models tables init
    m1_df, m2_df, m3_df = init_models(input_table)
    
    # Get and set the parameters of R|L distribution in models M1 and M3
    m1_df = get_L_dependent_dist_params(m1_df, 'R', l=0)
    m1_df = get_L_dependent_dist_params(m1_df, 'R', l=1)
    m3_df = get_L_dependent_dist_params(m3_df, 'R', l=0)
    m3_df = get_L_dependent_dist_params(m3_df, 'R', l=1)

    # Get and set the parameters of C|L distribution models M2 and M3
    m2_df = get_L_dependent_dist_params(m2_df, 'C', l=0)
    m2_df = get_L_dependent_dist_params(m2_df, 'C', l=1)
    m3_df = get_L_dependent_dist_params(m3_df, 'C', l=0)
    m3_df = get_L_dependent_dist_params(m3_df, 'C', l=1)

    # Calculate C|R and R|C parameters
    mu_r, var_r, mu_c, var_c, corr_r_c = get_columns_params(input_table['R'], input_table['C'])
    n = len(m1_df)
    for i in range(0, n): 
        # C|R
        r_i = m1_df['R'].iloc[i]
        mu, var = get_conditional_dist_params(mu_c, var_c, mu_r, var_r, corr_r_c, r_i)
        m1_df.iloc[i, m1_df.columns.get_loc('C|R_mean')] = mu
        m1_df.iloc[i, m1_df.columns.get_loc('C|R_var')] = var
        # R|C
        c_i = m2_df['C'].iloc[i]
        mu, var = get_conditional_dist_params(mu_r, var_r, mu_c, var_c, corr_r_c, c_i) 
        m2_df.iloc[i, m2_df.columns.get_loc('R|C_mean')] = mu
        m2_df.iloc[i, m2_df.columns.get_loc('R|C_var')] = var

    # Calculate normal dist. values for all models
    m1_df['P(R|L)'] = m1_df.apply(lambda x: norm_pdf(x['R'], x['R|L_mean'], x['R|L_var']), axis=1)
    m1_df['P(C|R)'] = m1_df.apply(lambda x: norm_pdf(x['C'], x['C|R_mean'], x['C|R_var']), axis=1)

    m2_df['P(C|L)'] = m2_df.apply(lambda x: norm_pdf(x['C'], x['C|L_mean'], x['C|L_var']), axis=1)
    m2_df['P(R|C)'] = m2_df.apply(lambda x: norm_pdf(x['R'], x['R|C_mean'], x['R|C_var']), axis=1)

    m3_df['P(R|L)'] = m3_df.apply(lambda x: norm_pdf(x['R'], x['R|L_mean'], x['R|L_var']), axis=1)
    m3_df['P(C|L)'] = m3_df.apply(lambda x: norm_pdf(x['C'], x['C|L_mean'], x['C|L_var']), axis=1)

    m1_df['P(L) * P(C|R) * P(R|L)'] = 0.5 * m1_df['P(C|R)'] * m1_df['P(R|L)']
    m2_df['P(L) * P(R|C) * P(C|L)'] = 0.5 * m2_df['P(R|C)'] * m2_df['P(C|L)']
    m3_df['P(L) * P(R|L) * P(C|L)'] = 0.5 * m3_df['P(C|L)'] * m3_df['P(R|L)']
    
    # Save as excel file
    m1_df.to_excel("likelihood_formulas_M1.xlsx")
    m2_df.to_excel("likelihood_formulas_M2.xlsx")
    m3_df.to_excel("likelihood_formulas_M3.xlsx")
    
    return m1_df, m2_df, m3_df

def get_LR(m1_df, m2_df, m3_df):
    L_m1 = m1_df['P(L) * P(C|R) * P(R|L)'].prod()
    L_m2 = m2_df['P(L) * P(R|C) * P(C|L)'].prod()
    L_m3 = m3_df['P(L) * P(R|L) * P(C|L)'].prod()
    likelihood_arr = [L_m1, L_m2, L_m3]

    # Get max likelihoos
    max_model = np.argmax([L_m1, L_m2, L_m3])
    others_models = [x for i,x in enumerate(range(0,3)) if i != max_model] 

    # Calculate the likelihood ratio
    LR = likelihood_arr[max_model] / max(likelihood_arr[others_models[0]], likelihood_arr[others_models[1]]) 
    return LR


def get_shuffled_df(df):
    """ Returns the given dataframe with 2 columns randomly shuffeled (L, R).
        No need to randomize the 3rd column """
    df['L'] = df['L'].sample(frac = 1).values
    df['R'] = df['R'].sample(frac = 1).values
    return df


In [20]:
# Choose a triplet L, R, C for the causality analysis
locus_idx = 241      # genotype
gene_exp_idx = 288 
phenotype_idx = 1842

locus = liver_genotypes.loc[[locus_idx]].replace({'B': 0, 'b': 0, 'D':1, 'H': np.nan, 'U': np.nan})[liver_common_bxd]
gene_exp = liver_matrix.iloc[[gene_exp_idx]][liver_common_bxd]
phenotype = liver_phenotypes.loc[[phenotype_idx]][liver_common_bxd]

# Create input table 
input_table = pd.concat([locus, gene_exp, phenotype]) # concate data types
input_table = input_table.T.reset_index()             # transpose
input_table.columns = ['Individual', 'L', 'R', 'C']
input_table = input_table.dropna()                    # ignore nulls
input_table = input_table.sort_values(by='L')         # sort by lucus (genotype 0 or 2)

In [21]:
m1_df, m2_df, m3_df = generate_models_df(input_table)
m1_df.head(5)

,Individual,L,R,C,R|L_mean,R|L_var,C|R_mean,C|R_var,P(R|L),P(C|R),P(L) * P(C|R) * P(R|L)
0,BXD1,0.0,0.7185,787.00,0.641457,0.346496,585.057,14697.1,0.671956,0.000822,0.000276
36,BXD85,0.0,1.2175,605.67,0.641457,0.346496,642.945,14697.1,0.419863,0.003139,0.000659
35,BXD8,0.0,0.8990,566.00,0.641457,0.346496,605.996,14697.1,0.615875,0.003116,0.000960
34,BXD77,0.0,0.6190,361.23,0.641457,0.346496,573.515,14697.1,0.677243,0.000710,0.000241
33,BXD73,0.0,1.2045,572.50,0.641457,0.346496,641.437,14697.1,0.428931,0.002799,0.000600


### Calculate The Models Likelihood and Likelihood Ratio:

Likelihood = L(θ;M) = p(data|θm)

In [22]:
LR = get_LR(m1_df, m2_df, m3_df)
print("The Likelihood-Ratio is: %.2f" % LR)

The Likelihood-Ratio is: 2.53


### Statistical estimation

In [23]:
n_tests = 100
LR_dist = []
for i in range(0,n_tests):
    shuffled_df = get_shuffled_df(input_table.copy())
    m1_df, m2_df, m3_df = generate_models_df(shuffled_df)
    LR_i = get_LR(m1_df, m2_df, m3_df)
    LR_dist.append(LR_i)

In [24]:
from scipy import stats 
import numpy as nupy
pval = stats.ttest_1samp(LR_dist, LR)[1]
if pval <= 0.05:
    print("Our LR is significant with pval ", pval)
else:
    print("Our LR is not significant ", pval)

Our LR is significant with pval  2.545549829959314e-10
